In [1]:
import pandas as pd
import utils
import random
import numpy as np

random.seed(9000)

In this notebook, both CellProfiler and DeepProfiler features of the plates are spherized and written to file. The whole plate normalized ORF, CRISPR and Compound profiles are read and the replicate plates are merged into a single dataframe. Then they are spherized and the output is written to file.

In [2]:
experiment_df = (
    pd.read_csv('output/experiment-metadata.tsv', sep='\t')
    .query('Batch=="2020_11_04_CPJUMP1" or Batch=="2020_11_04_CPJUMP1_DL"')
)

In [3]:
for batch in experiment_df.Batch.unique():
    batch_df = experiment_df.query('Batch==@batch')
    if batch == "2020_11_04_CPJUMP1":
        filename = "normalized.csv.gz"
    else:
        filename = "augmented.csv.gz"
    for cell in batch_df.Cell_type.unique():
        cell_df = batch_df.query('Cell_type==@cell')
        for perturbation in cell_df.Perturbation.unique():
            perturbation_df = cell_df.query('Perturbation==@perturbation')
            for timepoint in perturbation_df.Time.unique():
                timepoint_df = perturbation_df.query('Time==@timepoint')
                data_df = pd.DataFrame()
                for plate in timepoint_df.Assay_Plate_Barcode.unique():
                    plate_df = (
                        utils.load_data(batch, plate, filename)
                    )
                    data_df = utils.concat_profiles(data_df, plate_df)

                metadata_df = utils.get_metadata(data_df)
                features_df = utils.get_featuredata(data_df).replace(np.inf, np.nan).dropna(axis=1, how="any")
                data_df = pd.concat([metadata_df, features_df], axis=1)

                data_df = utils.sphere_plate_zca_corr(data_df)

                for plate in timepoint_df.Assay_Plate_Barcode.unique():
                    plate_df = data_df.query('Metadata_Plate==@plate')
                    data_df.to_csv(f'../profiles/{batch}/{plate}/{plate}_spherized.csv.gz', index=False, float_format='%.5g')
